In [ ]:
import xarray as xr
import numpy as np
import hydromt
import geopandas as gpd
from os.path import join
from hydromt.models import SfincsModel
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import colors, patheffects
import cartopy.crs as ccrs
# from matplotlib_scalebar.scalebar import ScaleBar


In [ ]:
rdir = '../../4_results'


In [ ]:
mod = SfincsModel(r'../../3_models/sfincs/00_base_riv', mode='r')
ds_sfx = mod.staticmaps

da_msk = ds_sfx['dep']==-9999

riv_mask = ds_sfx['rivmsk']
riv_mask.raster.set_nodata(0)
gdf_riv = riv_mask.where(~da_msk, 0).raster.vectorize()
gdf_region = mod.region

gdf_bnd = gpd.read_file(join(mod.root, 'gis', 'bnd.geojson')).set_index('index')
gdf_src = mod.forcing['dis'].vector.to_gdf().loc[[1,4],:]
gdf_src['label'] = ['Buzi', 'Pungwe']

In [ ]:
# read crs and utm zone > convert to cartopy
utm_zone = ds_sfx.raster.crs.to_wkt().split("UTM zone ")[1][:3]
utm = ccrs.UTM(int(utm_zone[:2]), "S" in utm_zone)
extent = np.array(ds_sfx.raster.box.total_bounds)[[0, 2, 1, 3]]
ylab = f"y coordinate UTM zone {utm_zone} [m]"
xlab = f"x coordinate UTM zone {utm_zone} [m]" 
ratio = ds_sfx.raster.ycoords.size / (ds_sfx.raster.xcoords.size * 1.2)
figsize = (10, 10 * ratio)


In [ ]:
#towns
df = pd.DataFrame({
    'Mefambisse': (-19.53935304519779, 34.629741259143714),
    'Buzi': (-19.883677, 34.599003),        
    'Beira': (-19.825007644673512, 34.86483377443984),    
}).T
df.columns = ['y', 'x']
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['x'], df['y']), crs=4326).to_crs(mod.crs)
gdf.to_file(r'../../1_data/towns.geojson', driver='GeoJSON')
gdf

In [ ]:
ann_kwargs = dict(
    xytext=(4, 0),
    textcoords="offset points",
    zorder=4,
    path_effects=[
        patheffects.Stroke(linewidth=3, foreground="w"),
        patheffects.Normal(),
    ],
)

In [ ]:
_, ax1 = mod.plot_basemap(fn_out=None, geoms=[], bmap='', shaded=True)

# da_msk.where(da_msk).plot(ax=ax1, cmap='ocean', add_colorbar=False, alpha=0.4, zorder=-1)
# da_msk.where(~da_msk).plot(ax=ax1, cmap='gray', add_colorbar=False, alpha=0.4, zorder=-1)
gdf_region.boundary.plot(ax=ax1, ls='-', lw=0.5, color='k', alpha=0.5, zorder=1)
extent = np.array(gdf_region.buffer(1000).total_bounds)[[0, 2, 1, 3]]
ax1.set_extent(extent, crs=utm)
gdf_bnd.plot(ax=ax1, marker="^", markersize=60, c="w", edgecolor="k", label='waterlevel boundary', zorder=4)
for label, grow in gdf_bnd.iterrows():
    x, y = grow.geometry.x, grow.geometry.y
    ax1.annotate(f'H{label}', xy=(x, y), **ann_kwargs)
gdf_src.plot(ax=ax1, marker=">", markersize=60, c="w", edgecolor="k", label='discharge boundary', zorder=4)
for _, grow in gdf_src.iterrows():
    x, y, name = grow.geometry.x, grow.geometry.y, grow.label
    ax1.annotate(f'Q {name}', xy=(x, y), **ann_kwargs)
ax1.text(0.7, 0.08, 'Mozambique Channel\n(Indian Ocean)', rotation=35, transform=ax1.transAxes)
gdf_riv.boundary.plot(ax=ax1, ls='--', lw=0.5, color='k', alpha=0.5, label='river outline')
 
# towns
gdf.plot(ax=ax1, marker='.', markersize=40, color="k", label='towns / cities', zorder=4)
for label, grow in gdf.iterrows():
    x, y = grow.geometry.x, grow.geometry.y
    ax1.annotate(f'{label}', xy=(x, y), **ann_kwargs)
  
ax1.legend(
    bbox_to_anchor=(1, 1),
    title="Legend",
    loc="upper right",
    frameon=True,
)

plt.savefig(join(rdir, f'sfincs_dep.png'), dpi=300, bbox_inches="tight")


In [ ]:
## hazard maps

In [ ]:
# hillshade background
res = 100
da = ds_sfx['dep']
ls = colors.LightSource(azdeg=210, altdeg=45)
hs = ls.hillshade(np.ma.masked_equal(da.values, -9999), vert_exag=25, dx=res, dy=res)

da_hs = xr.DataArray(
    dims=da.raster.dims, data=hs, coords=da.raster.coords
).where(riv_mask==0)
da_hs.plot(cmap='Greys', add_colorbar=False, alpha=0.2)

In [ ]:
postfix = ''
ds_flddph = hydromt.open_mfraster(join(rdir, 'hmax', f'*{postfix}.tif'))#.where(riv_mask==0)

In [ ]:
(ds_flddph[f'qb100_qp100_h100_p100'] - ds_flddph[f'qb100_qp000_h000_p000']).plot()

In [ ]:
fig, axs = plt.subplots(
    nrows=2, 
    ncols=2,
    figsize=(10,15),
    subplot_kw={'projection': utm},
    sharex = True, 
    sharey=True
)
rp, hmax = 50, 2.0
rp, hmax = 100, 3.0
# rp, hmax = 500, 3.0
maps = {
    f'qb000_qp000_h000_p{rp:03d}': f'{rp} year pluvial', 
    f'qb000_qp000_h{rp:03d}_p000': f'{rp} year coastal', 
    f'qb000_qp{rp:03d}_h000_p000': f'{rp} year fluvial Pungwe', 
    f'qb{rp:03d}_qp000_h000_p000': f'{rp} year fluvial Buzi'
}
hmin =0.15
axs = axs.flatten()
for i, (ax, key) in enumerate(zip(axs, maps)):

    da_hs.plot(ax=ax, cmap='Greys', add_colorbar=False, alpha=0.2)
    da_flddp = ds_flddph[key]
    cs = da_flddp.where(da_flddp>hmin).plot(ax=ax, cmap='viridis', vmin=hmin, vmax=hmax, add_colorbar=False)
    # gdf_riv.boundary.plot(ax=ax, ls='--', lw=0.5, color='k', alpha=0.5, label='river outline')
    ax.set_title(maps[key])
    
    if i % 2 == 0:
        ax.set_ylabel(ylab)    
        ax.yaxis.set_visible(True)
    else:
        ax.set_ylabel('')
        ax.yaxis.set_visible(False)
    if i // 2 == 1:
        ax.xaxis.set_visible(True)
        ax.set_xlabel(xlab)
    else:
        ax.xaxis.set_visible(False)
        ax.set_xlabel('')
        
    # towns
    for label, grow in gdf[:3].to_crs(mod.crs).iterrows():
        x, y = grow.geometry.x, grow.geometry.y
        ax.plot(x,y, '.k')
        xytext=(4,4) if label != 'Beira' else (5,-15)
        ann_kwargs.update(xytext=xytext)
        ax.annotate(f'{label}', xy=(x, y), **ann_kwargs)
    # rivers
    # for label, grow in gdf_src.iterrows():
    #     x, y, label = grow.geometry.x, grow.geometry.y, grow.label
    #     ax.annotate(f'{label}', xy=(x, y), xytext=(5,-5), **ann_kwargs)

# adjust ticks and whitespaces
ax.set_xticks(ax.get_xticks()[::2])
ax.set_extent(extent, crs=utm)
fig.subplots_adjust(wspace=0.05, hspace=0.06)
        
# # Draw the colorbar
cbar_ax = fig.add_axes([0.93, 0.33, 0.015, 0.3])
ticks = np.linspace(0,hmax,5)
ticks[0] = hmin
cbar=fig.colorbar(cs, cax=cbar_ax, orientation='vertical', ticks=ticks, extend='max', label='flood depth [m]')
cbar_ax.set_yticklabels(ticks)

# plt.savefig(join(rdir, f'flddph_rp{rp}{postfix}.png'), dpi=300, bbox_inches="tight")

In [ ]:
ds_flddph[key]

In [ ]:
dep, da_compound = 'full', ds_flddph[f'qb{rp:03d}_qp{rp:03d}_h{rp:03d}_p{rp:03d}'].squeeze(drop=True)
da_compound.name = 'flood depth'
da_compound.attrs.update(unit='m')
da_single_max = xr.concat([ds_flddph[key] for key in list(maps.keys())], dim='scen').max('scen')
flddff = da_compound - da_single_max
flddff.name = 'flood depth difference'
flddff.attrs.update(unit='m')

fig, axs = plt.subplots(
    nrows=1, 
    ncols=2,
    figsize=(10,8.5),
    subplot_kw={'projection': utm},
    sharex = True, 
    sharey=True
)
dh=0.01
cbar_kwargs = dict(orientation='horizontal', anchor=(0.5,0), pad=0.005, shrink=0.7)

ax = axs[0]
da_hs.plot(ax=ax, cmap='Greys', add_colorbar=False, alpha=0.2)
da_compound.where(da_compound>hmin).plot(ax=ax, cmap='viridis', vmin=hmin, vmax=hmax, add_colorbar=True, cbar_kwargs=cbar_kwargs)
ax.set_title(f'{rp} year compound ({dep} dependence)')
ax.xaxis.set_visible(True)
ax.yaxis.set_visible(True)
ax.set_ylabel(ylab)
ax.set_xlabel(xlab)

# towns
for label, grow in gdf[:3].to_crs(mod.crs).iterrows():
    x, y = grow.geometry.x, grow.geometry.y
    ax.plot(x,y, '.k')
    xytext=(4,4) if label != 'Beira' else (5,-15)
    ann_kwargs.update(xytext=xytext)
    ax.annotate(f'{label}', xy=(x, y), **ann_kwargs)

ax = axs[1]
da_hs.plot(ax=ax, cmap='Greys', add_colorbar=False, alpha=0.2)
flddff.where(np.abs(flddff)>=dh).plot(ax=ax, cmap='coolwarm', vmin=-0.2, vmax=0.2, add_colorbar=True, cbar_kwargs=cbar_kwargs)
ax.set_title(f'{rp} year compound minus max. single driver')
ax.xaxis.set_visible(True)
ax.yaxis.set_visible(True)
ax.set_ylabel('')
ax.set_xlabel(xlab)

# towns
for label, grow in gdf[:3].to_crs(mod.crs).iterrows():
    x, y = grow.geometry.x, grow.geometry.y
    ax.plot(x,y, '.k')
    xytext=(4,4) if label != 'Beira' else (5,-15)
    ann_kwargs.update(xytext=xytext)
    ax.annotate(f'{label}', xy=(x, y), **ann_kwargs)

# adjust ticks and whitespaces
ax.set_xticks(ax.get_xticks()[::2])
ax.set_extent(extent, crs=utm)
ax.yaxis.set_visible(False)
fig.subplots_adjust(wspace=0.05, hspace=0.06)

# plt.savefig(join(rdir, f'flddph_compound_{dep}dep_rp{rp}{postfix}.png'), dpi=300, bbox_inches="tight")